# Training the model using a sequential neural network with 8 layers

In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import tensorflow as tf
import os
from datetime import datetime

# Directory for storing the TensorBoard logs
logs_directory = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=logs_directory, histogram_freq=1)

# List all physical devices TensorFlow can access
physical_devices = tf.config.list_physical_devices()
print("Available devices:")
for device in physical_devices:
    print(device)

# Checking for GPU availability
if any("GPU" in str(device.device_type) for device in physical_devices):
    print("Using Mac GPU via TensorFlow Metal plugin.")
else:
    print("GPU not available, using CPU instead.")

# Load your dataset
df = pd.read_csv("088_238_440_644_PBMRS_Pre_Processed_Data.csv")

# Encode the 'Genre' column into numerical values
label_encoder = LabelEncoder()
df['GenreEncoded'] = label_encoder.fit_transform(df['Genre Names'])

# Text preprocessing for Word2Vec
class TextCleaner(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.apply(lambda x: x.split())  # Tokenization

# Preprocessing for other features
preprocessor = ColumnTransformer(transformers=[
    ('origin', Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]), ['Origin'])
])

# Word2Vec model
word2vec_model = Word2Vec(sentences=df['overview'].apply(lambda x: x.split()), vector_size=100, window=5, min_count=1, workers=4)

# Tokenize and pad sequences for neural network input
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['overview'])
sequences = tokenizer.texts_to_sequences(df['overview'])
padded_sequences = pad_sequences(sequences)

# Neural network model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=padded_sequences.shape[1]))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(df['Genre Names'].unique()), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df['GenreEncoded'], test_size=0.2, random_state=42)

# Train the model with the TensorBoard callback
model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])

# Evaluate model accuracy
accuracy = model.evaluate(X_test, y_test, verbose=0)[1]
print(f"Model Accuracy: {accuracy:.2f}")

# Function to predict the genre for a new plot
def predict_genre_nn(new_plot):
    new_sequence = tokenizer.texts_to_sequences([new_plot])
    new_padded_sequence = pad_sequences(new_sequence, maxlen=padded_sequences.shape[1])
    prediction_probs = model.predict(new_padded_sequence)[0]
    predicted_genre_index = np.argmax(prediction_probs)
    predicted_genre = label_encoder.inverse_transform([predicted_genre_index])[0]
    print("Confidence of Each Genre:")
    for genre_index, prob in enumerate(prediction_probs):
        genre = label_encoder.inverse_transform([genre_index])[0]
        print(f"{genre}: {prob:.2%}")
    return predicted_genre, prediction_probs[predicted_genre_index]


Available devices:
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Using Mac GPU via TensorFlow Metal plugin.


2024-04-17 10:25:31.119636: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-04-17 10:25:31.119655: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-04-17 10:25:31.119663: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-04-17 10:25:31.119709: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-17 10:25:31.120012: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/100


2024-04-17 10:25:31.824413: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


60/60 [==============================] - 5s 77ms/step - loss: 6.3898 - accuracy: 0.0474 - val_loss: 5.8906 - val_accuracy: 0.0865
Epoch 2/100
60/60 [==============================] - 4s 71ms/step - loss: 5.8193 - accuracy: 0.0677 - val_loss: 5.9031 - val_accuracy: 0.0865
Epoch 3/100
60/60 [==============================] - 4s 69ms/step - loss: 5.6330 - accuracy: 0.0698 - val_loss: 5.8792 - val_accuracy: 0.0865
Epoch 4/100
60/60 [==============================] - 4s 69ms/step - loss: 5.3149 - accuracy: 0.0690 - val_loss: 5.9225 - val_accuracy: 0.0917
Epoch 5/100
60/60 [==============================] - 4s 65ms/step - loss: 5.0851 - accuracy: 0.0706 - val_loss: 6.0120 - val_accuracy: 0.0875
Epoch 6/100
60/60 [==============================] - 4s 66ms/step - loss: 4.9706 - accuracy: 0.0755 - val_loss: 6.1755 - val_accuracy: 0.0885
Epoch 7/100
60/60 [==============================] - 4s 64ms/step - loss: 4.8224 - accuracy: 0.0920 - val_loss: 6.3448 - val_accuracy: 0.0500
Epoch 8/100
60/60 

# Saving the Model trained in the earlier code block

In [44]:
import joblib

model.save('model.h5')  # saves the architecture and weights
import pickle

# Saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Saving label encoder
joblib.dump(label_encoder, 'label_encoder.joblib')
word2vec_model.save("word2vec.model")

'model.save(\'model.h5\')  # saves the architecture and weights\nimport pickle\n\n# Saving tokenizer\nwith open(\'tokenizer.pickle\', \'wb\') as handle:\n    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)\nimport joblib\n\n# Saving label encoder\njoblib.dump(label_encoder, \'label_encoder.joblib\')\nword2vec_model.save("word2vec.model")'